## osumapper #4: New Map Reader


Set the input file string to a timed (having the right BPM/offset) .osu file.

It converts the map/music to Python readable format.

In [113]:
#from act_newmap_prep import *

# input file here! (don't remove the "r" before string)
# file_path = r'..\..\test_data\test.osu'

# Or use auto timing with music file only!!

from act_timing import *;
music_path = r"..\..\test_data\audio.mp3"
file_path = get_timed_osu_file(music_path, game_mode=0);

step4_read_new_map(file_path);

Timing Analyzer v0.32.4 alpha

Reading audio file...
***************************************************************************
         Calculation results may be incorrect.                             
         Contact the author of this program for more information.          
***************************************************************************
Processing audio data...
Calculating BPM...
Calculating offset...

INFO: Portions marked with '#' in the timeline probably need additional    
          timing section(s).                                               
INFO: Offset of portions marked with '+' or '-' probably have slightly     
          different offset than the given timing or calculation result.    

Timeline   >>   +      +    ## ### <<
Time (min) >>0     1     2     3   <<


Timing Analyzer v0.32.4 alpha

Reading audio file...
***************************************************************************
         Calculation results may be incorrect.                  

## osumapper #5: Rhythm Predictor

Calculates a map's rhythm based on the music and timing.

Parameters:

"note_density" determines how many notes will be placed on the timeline, ranges from 0 to 1.<br>
"slider_favor" determines how the model favors sliders against circles, ranges from -1 to 1.<br>
"dist_multiplier" determines the distance snap. ranges from 0 to +∞. Of course 0/+∞ are not advisable.<br>
"divisor_favor" determines how the model favors notes to be on X divisors starting from a beat (white, blue, red, blue), ranges from -1 to 1 each.<br>
"slider_max_ticks" determines the max amount of time a slider can slide, ranges from 1 to +∞.

In [114]:
from act_rhythm_calc import *

model = step5_load_model();
npz = step5_load_npz();
params = step5_set_params(dist_multiplier=1, note_density=0.75, slider_favor=0.1, divisor_favor=[0] * 4, slider_max_ticks=8);

predictions = step5_predict_notes(model, npz, params);
converted = step5_convert_sliders(predictions, params);

1703 notes predicted.


In [115]:
step5_save_predictions(converted);

## osumapper #6: Map flow generator

Generate the final map using a Generative Adversarial Network.

Parameters:

- note_distance_basis: the baseline for distance snap between notes
- max_ticks_for_ds: max number of time ticks (each 1/4) that it uses the distance snap
- next_from_slider_end: use slider end instead of slider head for calculating distance
- box_loss_border, box_loss_value: it's like a barrier on the map edges that bounces off the circles
- divisor, note_group_size: don't change unless you're using a special model built for it
- good_epoch, max_epoch: controls the training time. less time makes it faster but risks less quality
- g_\*, c_\*: hyperparameters used by GAN. No one knows how they work but they mysterically affect the result

In [116]:
from act_gan import *;

gan_params = {
    "divisor" : 4,
    "good_epoch" : 12,
    "max_epoch" : 30,
    "note_group_size" : 10,
    "g_epochs" : 1,
    "c_epochs" : 1,
    "g_batch" : 25, #50
    "g_input_size" : 100, #50
    "c_true_batch" : 120, # 140
    "c_false_batch" : 5, # 5
    "c_randfalse_batch" : 5,
    "note_distance_basis" : 350,
    "next_from_slider_end" : False,
    "max_ticks_for_ds" : 16, # 8 oder 4
    "box_loss_border" : 0.1,
    "box_loss_value" : 0.4,
    "box_loss_weight" : 1
};

step6_set_gan_params(gan_params);
osu_a, data = step6_run_all();

# of groups: 136
Group 0, Epoch 1: G loss: 0.7549962401390076 vs. C loss: 0.18041469156742096
Group 0, Epoch 2: G loss: 0.4000888466835022 vs. C loss: 0.08816941827535629
Group 0, Epoch 3: G loss: 0.3083866834640503 vs. C loss: 0.09187985211610794
Group 0, Epoch 4: G loss: 0.2470843344926834 vs. C loss: 0.09187737852334976
Group 0, Epoch 5: G loss: 0.30016496777534485 vs. C loss: 0.0877106636762619
Group 0, Epoch 6: G loss: 0.4133431315422058 vs. C loss: 0.07697819918394089
Group 0, Epoch 7: G loss: 0.7006438970565796 vs. C loss: 0.06097709760069847
Group 0, Epoch 8: G loss: 0.9360105395317078 vs. C loss: 0.06579714268445969
Group 0, Epoch 9: G loss: 1.1069200038909912 vs. C loss: 0.051713164895772934
Group 0, Epoch 10: G loss: 1.2219347953796387 vs. C loss: 0.0524318628013134
Group 0, Epoch 11: G loss: 1.4339802265167236 vs. C loss: 0.0498473234474659
Group 0, Epoch 12: G loss: 1.39924955368042 vs. C loss: 0.022647593170404434
Group 0, Epoch 13: G loss: 1.0162347555160522 vs. C loss: 

Group 3, Epoch 26: G loss: 1.1258987188339233 vs. C loss: 0.019762370735406876
Group 3, Epoch 27: G loss: 1.0931227207183838 vs. C loss: 0.010421101935207844
Group 3, Epoch 28: G loss: 1.034908413887024 vs. C loss: 0.0024400765541940928
Group 3, Epoch 29: G loss: 1.1403822898864746 vs. C loss: 0.026652740314602852
Group 3, Epoch 30: G loss: 1.123765468597412 vs. C loss: 0.01853817328810692
Group 4, Epoch 1: G loss: 0.7138016819953918 vs. C loss: 0.209774911403656
Group 4, Epoch 2: G loss: 0.3044179677963257 vs. C loss: 0.09379390627145767
Group 4, Epoch 3: G loss: 0.14966228604316711 vs. C loss: 0.09263025224208832
Group 4, Epoch 4: G loss: 0.11812211573123932 vs. C loss: 0.0944635346531868
Group 4, Epoch 5: G loss: 0.09511815756559372 vs. C loss: 0.09484349936246872
Group 4, Epoch 6: G loss: 0.07630432397127151 vs. C loss: 0.09465108811855316
Group 4, Epoch 7: G loss: 0.09050498157739639 vs. C loss: 0.09481889754533768
Group 4, Epoch 8: G loss: 0.10410384833812714 vs. C loss: 0.094734

Group 11, Epoch 7: G loss: 0.6573793888092041 vs. C loss: 0.05054273456335068
Group 11, Epoch 8: G loss: 0.7501370906829834 vs. C loss: 0.03551500290632248
Group 11, Epoch 9: G loss: 0.7729458808898926 vs. C loss: 0.020383479073643684
Group 11, Epoch 10: G loss: 0.8296344876289368 vs. C loss: 0.018223507329821587
Group 11, Epoch 11: G loss: 0.8779327273368835 vs. C loss: 0.017196668311953545
Group 11, Epoch 12: G loss: 0.9541052579879761 vs. C loss: 0.019964806735515594
Group 11, Epoch 13: G loss: 0.9315236806869507 vs. C loss: 0.005806731525808573
Group 12, Epoch 1: G loss: 0.6128122210502625 vs. C loss: 0.21734870970249176
Group 12, Epoch 2: G loss: 0.2954385280609131 vs. C loss: 0.0964922085404396
Group 12, Epoch 3: G loss: 0.1465964913368225 vs. C loss: 0.0926547423005104
Group 12, Epoch 4: G loss: 0.11081884801387787 vs. C loss: 0.09307420998811722
Group 12, Epoch 5: G loss: 0.11155486106872559 vs. C loss: 0.09168010205030441
Group 12, Epoch 6: G loss: 0.1228569895029068 vs. C los

Group 17, Epoch 4: G loss: 0.12010098993778229 vs. C loss: 0.0907301977276802
Group 17, Epoch 5: G loss: 0.12315727770328522 vs. C loss: 0.08633380383253098
Group 17, Epoch 6: G loss: 0.14685165882110596 vs. C loss: 0.08339887112379074
Group 17, Epoch 7: G loss: 0.35432618856430054 vs. C loss: 0.08208096772432327
Group 17, Epoch 8: G loss: 0.4167746901512146 vs. C loss: 0.08012501895427704
Group 17, Epoch 9: G loss: 0.5311318635940552 vs. C loss: 0.06916610151529312
Group 17, Epoch 10: G loss: 0.6080955862998962 vs. C loss: 0.06469674408435822
Group 17, Epoch 11: G loss: 0.5614209771156311 vs. C loss: 0.05468836799263954
Group 17, Epoch 12: G loss: 0.6109597682952881 vs. C loss: 0.04526584967970848
Group 17, Epoch 13: G loss: 0.7475728392601013 vs. C loss: 0.03927276283502579
Group 18, Epoch 1: G loss: 0.5775355696678162 vs. C loss: 0.23113712668418884
Group 18, Epoch 2: G loss: 0.2438724786043167 vs. C loss: 0.09280554205179214
Group 18, Epoch 3: G loss: 0.0990346372127533 vs. C loss:

Group 24, Epoch 3: G loss: 0.10572457313537598 vs. C loss: 0.0915098637342453
Group 24, Epoch 4: G loss: 0.07864175736904144 vs. C loss: 0.09366187453269958
Group 24, Epoch 5: G loss: 0.08002801239490509 vs. C loss: 0.09379785507917404
Group 24, Epoch 6: G loss: 0.072266586124897 vs. C loss: 0.09339568018913269
Group 24, Epoch 7: G loss: 0.10030323266983032 vs. C loss: 0.09165715426206589
Group 24, Epoch 8: G loss: 0.09255094826221466 vs. C loss: 0.08976192027330399
Group 24, Epoch 9: G loss: 0.11550231277942657 vs. C loss: 0.08544917404651642
Group 24, Epoch 10: G loss: 0.17691129446029663 vs. C loss: 0.07720475643873215
Group 24, Epoch 11: G loss: 0.2698649764060974 vs. C loss: 0.0676681399345398
Group 24, Epoch 12: G loss: 0.3860322833061218 vs. C loss: 0.06036511808633804
Group 24, Epoch 13: G loss: 0.43195047974586487 vs. C loss: 0.05642030015587807
Group 25, Epoch 1: G loss: 0.614478588104248 vs. C loss: 0.21498125791549683
Group 25, Epoch 2: G loss: 0.24100157618522644 vs. C los

...

Group 104, Epoch 2: G loss: 0.1701488494873047 vs. C loss: 0.09147562086582184
Group 104, Epoch 3: G loss: 0.09020928293466568 vs. C loss: 0.0931183248758316
Group 104, Epoch 4: G loss: 0.06739531457424164 vs. C loss: 0.09284727275371552
Group 104, Epoch 5: G loss: 0.0688643753528595 vs. C loss: 0.09448332339525223
Group 104, Epoch 6: G loss: 0.05684930831193924 vs. C loss: 0.09436607360839844
Group 104, Epoch 7: G loss: 0.05264843627810478 vs. C loss: 0.09425713121891022
Group 104, Epoch 8: G loss: 0.05092736333608627 vs. C loss: 0.0942666158080101
Group 104, Epoch 9: G loss: 0.04855019226670265 vs. C loss: 0.09373889863491058
Group 104, Epoch 10: G loss: 0.05233673006296158 vs. C loss: 0.09289685636758804
Group 104, Epoch 11: G loss: 0.062467724084854126 vs. C loss: 0.09047539532184601
Group 104, Epoch 12: G loss: 0.0777888149023056 vs. C loss: 0.08761931955814362
Group 105, Epoch 1: G loss: 0.5935565233230591 vs. C loss: 0.2227843701839447
Group 105, Epoch 2: G loss: 0.3103072941303

Group 111, Epoch 3: G loss: 0.07723093032836914 vs. C loss: 0.09349807351827621
Group 111, Epoch 4: G loss: 0.05283802002668381 vs. C loss: 0.09450443089008331
Group 111, Epoch 5: G loss: 0.055961716920137405 vs. C loss: 0.09495122730731964
Group 111, Epoch 6: G loss: 0.04991573095321655 vs. C loss: 0.09513244032859802
Group 111, Epoch 7: G loss: 0.042956314980983734 vs. C loss: 0.09512211382389069
Group 111, Epoch 8: G loss: 0.030762095004320145 vs. C loss: 0.0949249342083931
Group 111, Epoch 9: G loss: 0.030781269073486328 vs. C loss: 0.09463201463222504
Group 111, Epoch 10: G loss: 0.0266486294567585 vs. C loss: 0.09437358379364014
Group 111, Epoch 11: G loss: 0.020987655967473984 vs. C loss: 0.09383786469697952
Group 111, Epoch 12: G loss: 0.026642069220542908 vs. C loss: 0.09286253899335861
Group 112, Epoch 1: G loss: 0.5681986808776855 vs. C loss: 0.2326892912387848
Group 112, Epoch 2: G loss: 0.23973429203033447 vs. C loss: 0.09188796579837799
Group 112, Epoch 3: G loss: 0.11206

Group 117, Epoch 25: G loss: 0.9147981405258179 vs. C loss: 0.04068666696548462
Group 117, Epoch 26: G loss: 0.8162051439285278 vs. C loss: 0.055447038263082504
Group 117, Epoch 27: G loss: 0.6699450016021729 vs. C loss: 0.08313664048910141
Group 117, Epoch 28: G loss: 0.20166358351707458 vs. C loss: 0.0690072774887085
Group 117, Epoch 29: G loss: 0.1451880782842636 vs. C loss: 0.07064071297645569
Group 117, Epoch 30: G loss: 0.11901652812957764 vs. C loss: 0.06990335136651993
Group 118, Epoch 1: G loss: 0.4874822497367859 vs. C loss: 0.23270007967948914
Group 118, Epoch 2: G loss: 0.25399577617645264 vs. C loss: 0.09963840991258621
Group 118, Epoch 3: G loss: 0.17971684038639069 vs. C loss: 0.09053079783916473
Group 118, Epoch 4: G loss: 0.17132681608200073 vs. C loss: 0.08454246073961258
Group 118, Epoch 5: G loss: 0.22017315030097961 vs. C loss: 0.08274343609809875
Group 118, Epoch 6: G loss: 0.26592540740966797 vs. C loss: 0.08546119183301926
Group 118, Epoch 7: G loss: 0.343078106

Group 125, Epoch 7: G loss: 0.03743783384561539 vs. C loss: 0.095186248421669
Group 125, Epoch 8: G loss: 0.03382181376218796 vs. C loss: 0.09512849897146225
Group 125, Epoch 9: G loss: 0.035337094217538834 vs. C loss: 0.09525064378976822
Group 125, Epoch 10: G loss: 0.03169066458940506 vs. C loss: 0.09536687284708023
Group 125, Epoch 11: G loss: 0.031801603734493256 vs. C loss: 0.09511527419090271
Group 125, Epoch 12: G loss: 0.0414251983165741 vs. C loss: 0.09483318775892258
Group 125, Epoch 13: G loss: 0.03712957352399826 vs. C loss: 0.09473864734172821
Group 125, Epoch 14: G loss: 0.03446463868021965 vs. C loss: 0.09416718035936356
Group 125, Epoch 15: G loss: 0.03579123318195343 vs. C loss: 0.0937587320804596
Group 125, Epoch 16: G loss: 0.044298335909843445 vs. C loss: 0.09244000166654587
Group 126, Epoch 1: G loss: 0.5628087520599365 vs. C loss: 0.21454809606075287
Group 126, Epoch 2: G loss: 0.2990493178367615 vs. C loss: 0.09804228693246841
Group 126, Epoch 3: G loss: 0.161436

Group 132, Epoch 3: G loss: 0.1382649838924408 vs. C loss: 0.08938883244991302
Group 132, Epoch 4: G loss: 0.09465259313583374 vs. C loss: 0.09081239253282547
Group 132, Epoch 5: G loss: 0.11699269711971283 vs. C loss: 0.08716560900211334
Group 132, Epoch 6: G loss: 0.16681605577468872 vs. C loss: 0.08288736641407013
Group 132, Epoch 7: G loss: 0.367477685213089 vs. C loss: 0.05789528787136078
Group 132, Epoch 8: G loss: 0.595689594745636 vs. C loss: 0.05585567280650139
Group 132, Epoch 9: G loss: 0.6959965825080872 vs. C loss: 0.03613853454589844
Group 132, Epoch 10: G loss: 0.6042409539222717 vs. C loss: 0.03330691158771515
Group 132, Epoch 11: G loss: 0.6752161979675293 vs. C loss: 0.03186707943677902
Group 132, Epoch 12: G loss: 0.7810377478599548 vs. C loss: 0.02575569413602352
Group 133, Epoch 1: G loss: 0.5989699363708496 vs. C loss: 0.2340313047170639
Group 133, Epoch 2: G loss: 0.24939222633838654 vs. C loss: 0.0990968868136406
Group 133, Epoch 3: G loss: 0.11856603622436523 v

### Since the generation will take a while...

we can appreciate a nice picture of Cute Sophie!!

<img src="https://i.imgur.com/Ko2wogO.jpg" />

Do a little modding to the map.

Parameters:

- stream_regularizer: fix bad streams. integer for modes (0,1,2,3,4) 0=inactive
- slider_mirror: mirror slider ends if they go outside map area. (0,1) 0=inactive 1=active

In [117]:
from act_modding import *

modding_params = {
    "stream_regularizer" : 1,
    "slider_mirror" : 1
}

osu_a, data = step7_modding(osu_a, data, modding_params);

Finally, save the data into an .osu file!

In [118]:
from act_final import *

saved_osu_name = step8_save_osu_file(osu_a, data);

# for taiko mode only (comment out the above line and use below)
# from act_taiko_hitsounds import *
# taiko_hitsounds_params = step8_taiko_hitsounds_set_params(divisor=4, metronome_count=4)
# hitsounds = step8_apply_taiko_hitsounds(osu_a, data, params=taiko_hitsounds_params)
# saved_osu_name = step8_save_osu_file(osu_a, data, hitsounds=hitsounds);

# clean up the folder
step8_clean_up();

success! built osu map from json data.
elapsed time: 0.073 s

finished on: 2021-11-14 17:38:36.260502


If it works alright, you should have a nice .osu file under the folder of these notebooks now!

If it does not work, please tell me the problem so probably I could fix it!

For bug reports and feedbacks either report it on github or use discord: <br>
[https://discord.com/invite/npmSy7K](https://discord.com/invite/npmSy7K)